In [4]:
import requests
import xmltodict
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from dotenv import load_dotenv
import os
import torch
import time
from tqdm import tqdm
import torch
import torch.nn.functional as F
import random

In [2]:
embedding_file = "embeddings.pt"
existing_embeddings = torch.load(embedding_file)
papers = pd.read_json('papers.json', orient='records', lines=True)
questions = pd.read_json('questions.json', orient='records', lines=True).T

C:\Users\pilug\AppData\Local\Temp\ipykernel_17936\2889146714.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  existing_embeddings = torch.load(embedding_file)


In [23]:
triplets = []

def create_training_pair(question, paper_embedding, irrelevant_embedding):
    return {
            "question": question,
            "positive": paper_embedding.tolist(),
            "negative": irrelevant_embedding.tolist()
        }

def get_irrelevant_embedding(paper_embedding):
    existing_embeddings_tensor = torch.stack(existing_embeddings)  # Assuming each element in the list is a tensor
    cos_similarities = F.cosine_similarity(paper_embedding, existing_embeddings_tensor)

    similarity_threshold = 0.3  # You can adjust this threshold to determine how "distant" the papers should be

    indices_below_threshold = torch.where(cos_similarities < similarity_threshold)[0]
    return indices_below_threshold

for i in tqdm(range(len(papers))):
    paper_embedding = existing_embeddings[i]
    irrelevant_embeddings = get_irrelevant_embedding(paper_embedding)
    for q in questions.iloc[i][0]['0']['0'].split('\n'):
        if ':' in q: 
            question = q.split(':')[1].strip()
            idx = random.choice(irrelevant_embeddings)
            irrelevant = paper_embedding[idx]

            create_training_pair(question, paper_embedding, irrelevant)

Chemical engineering of CRISPR-Cas systems for therapeutic application.
CRISPR/Cas9-Engineering for Increased Amylolytic Potential of Microbes for Sustainable Wastewater Treatment: A Review.
Bidirectional Prime Editing: Combining Precision with Versatility for Genome Editing.
Automated plasmid design for marker-free genome editing in budding yeast.
Chiral nanosystem and chiral supraparticles for drug delivery: an expert opinion.
Increasing Rubisco as a simple means to enhance photosynthesis and productivity now without lowering nitrogen use efficiency.
Anionic polymer coating for enhanced delivery of Cas9 mRNA and sgRNA nanoplexes.


IndexError: list index out of range